In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_circles, make_blobs
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# Support Vector Machines (SVMs) - Tutorial - 17.06.2021

## Overview

Hello! This is a practical tutorial. In this notebook, we are going to explore Support Vector Machines (SVMs) hands-on.  We have left a few `#TODO` where implementation is missing. Take a look and try to implement it yourself! HINT: There is a comprehensive scikit-learn documentation on SVMs [here](https://scikit-learn.org/stable/modules/svm.html#svm-classification). We will illustrate the implementation at the tutorial on Thursday. See you there!

The main goals of this notebook are:

- Learn how to use the plug-and-play models implemented in the Python package scikit-learn.
- Explore the power of the parameter `C`.
- Explore the power of different kernels with different parameters `gamma`.

## Datasets
In this notebook we will be using the following three datasets:
- Dataset A: 2-dimensional 2-classes isotropic Gaussian blobs. The samples for each class are generated from a gaussian distribution with a certain mean and standard deviation. This dataset is linearly separable.
- Dataset B: 2-dimensional 2-classes isotropic Gaussian blobs. The samples for each class are generated from a gaussian distribution with a certain mean and standard deviation. This dataset is not linearly separable.
- Dataset C: 2-dimensional 2-classes circles.  Samples for each class are generated from circles with a certain radius plus some noise. This dataset is not linearly separable.
- Dataset D: 2-dimensional 3-classes isotropic Gaussian blobs. The samples for each class are generated from a gaussian distribution with a certain mean and standard deviation. This dataset is linearly separable.

In [ ]:
# function to plot a dataset
def plot_dataset(X, y, title, axes=None):
      if axes is None:
        axes = plt.gca()
      y_zero = y==0
      y_one = y==1

      axes.scatter(X[y_zero,0],X[y_zero,1],color = "red", label = "Class 0")
      axes.scatter(X[y_one,0],X[y_one,1], color = "blue", label = "Class 1")
      axes.set_title(title)
      axes.set_xlabel("x1")
      axes.set_ylabel("x2")
      axes.legend()

In [ ]:
# Fix seed
np.random.seed(5)
# Number of training samples
n_samples = 100
# Dataset A
X_blobs1, y_blobs1 = make_blobs(n_samples=n_samples, centers=[(-1, -1), (1, 1)], cluster_std = 0.45)
X_blobs1_test, y_blobs1_test = make_blobs(n_samples=1000, centers=[(-1, -1), (1, 1)], cluster_std = 0.45)
# Dataset B
X_blobs2, y_blobs2 = make_blobs(n_samples=2*n_samples, centers=[(-1, -1), (1, 1)], cluster_std = 0.8)
X_blobs2_test, y_blobs2_test = make_blobs(n_samples=500, centers=[(-1, -1), (1, 1)], cluster_std = 0.8)
# Dataset C
X_circles, y_circles = make_circles(n_samples=n_samples,factor=.1, noise=.05)
X_circles_test, y_circles_test = make_circles(n_samples=500,factor=.1, noise=.05)
# Dataset D
X_blobs3, y_blobs3 = make_blobs(n_samples=n_samples, centers=[(-1, -1), (1, 1), (2,-1)], cluster_std = 0.5)
X_blobs3_test, y_blobs3_test = make_blobs(n_samples=500, centers=[(-1, -1), (1, 1), (2,-1)], cluster_std = 0.5)

In [ ]:
# Ploting the datasets
fig, ax = plt.subplots(1,4, figsize=(20,5))
# 2 classes
plot_dataset(X_blobs1, y_blobs1, title="Dataset A: linearly separable", axes = ax[0])
plot_dataset(X_blobs2, y_blobs2, title="Dataset B: not linearly separable", axes = ax[1])
plot_dataset(X_circles, y_circles, title="Dataset C: non-linear", axes = ax[2])
# 3 classes
y_two = y_blobs3==2
ax[3].scatter(X_blobs3[y_two,0], X_blobs3[y_two,1],color = "green", label = "Class 2")
plot_dataset(X_blobs3, y_blobs3, title="Dataset C: non-linear", axes = ax[3])


## How to
Throughout this notebook we will be analyzing SVM classifiers with different configurations (i.e., parameters and kernels) by computing the test error of the classifier(s), plotting their classification boundary and outputting the decision function. Lets start with looking at the functions that perfom this:

- Compute test error with a function called `test_error`. Use as input the ground truth (correct) labels `y_true` and the predicted labels, as returned by a classifier `y_pred`. 

In [ ]:
def test_error(y_true, y_pred):
    #TODO
    return score

- Plot the decision boundary of a trained 2D Support Vector Classifier (SVC) (`model`) with a function called `plot_svc_decision_function`. The function should also plot the margin and have the option to plot the support vectors (`plot_support`). 

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:            #If no figure handle is provided, it opens the current figure
        ax = plt.gca()
        
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)    #30 points in the grid axis
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)                 # We create a grid with the x,y coordinates defined above
    
    # From the grid to a list of (x,y) values. 
    # Check Numpy help for ravel()
    
    xy = np.vstack([X.ravel(), Y.ravel()]).T 
    
    
    # get decision boundary
    #TODO
    
    # plot decision boundary and margins
    #TODO

    
    # plot support vectors
    if plot_support:
        #TODO
        
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

# 1. Linearly separable data
We start with looking at dataset A. 
## Maximum Margin Hyperplane
You have the following three hyperplanes/decision boundaries given. You can see them as three suggestions on how to take decisions on the given dataset. Can you tell, which suggestion is the best? How would you proceed to analyze them? Lets take a look at it together, step by step. 

A maximum margin hyperplane $(w, b)$ for a linearly separable set of training data $\left(x_{i}, y_{i}\right)_{i=1}^{n}$ is defined as
$$
\max _{\mathbf{w} \in \mathbb{R}^{d}, b \in \mathbb{R}} \min \left\{\left\|x-x_{i}\right\| \mid\langle w, x\rangle+b=0, x \in \mathbb{R}^{d}, i=1, \ldots, n\right\}
$$
where we optimize over all $(w, b)$ such that $y_{i}\left(\left\langle w, x_{i}\right\rangle+b\right)>0$.


A linear classifier is determined by the weight vector ${w}$ and the offset $b$.
$$
\hat{y}({x})=\operatorname{sign}(\langle{w}, {x}\rangle+b)
$$

These are our three hyperplanes (classifiers) for the 2D dataset.
- Classifier A: $w = (1, 1)$, $b = 0$
- Classifier B: $w=(1, 0.3)$, $b= 0$
- Classifier C: $w = (0.1, 1.6)$, $b = 0$

We will now take a closer look at maximum magine hyperplanes.

- 1.1 Lets begin with plotting, how the given hyperplanes split the training data. Create a function `get_decision boundary` that computes the decision boundary given weights $w$ and bias $b$ for a given vector `X`. Then print all three hyperplanes. What do you observe? Can you tell from this plot, which one is the best?

In [ ]:
# weights, bias as given
# TODO

def get_decision_boundary(w,b, X):
    # TODO

# plot all three decision boundaries
# TODO

# plot dataset
# TODO


- 1.2 Assume a fourth candidate D shows up with $w=(2,2)$ and $b = 0$. Before you plot it, can you say, if it will be better or worse than the other candidates? Now plot it together with the other candidates, what do you observe?

In [ ]:
# weights, bias as given 
# TODO

# plot decision boundary
# TODO

# plot dataset 
# TODO

- 1.3 How do we find the best classifier candidate? From the lecture, we know that a maximum margin hyperplane is a hyperplane which correctly classifies the data and has maximum distance/margin to the data. Can you draw around each decision boundary a *margin* with some width up to the nearest point? What do you observe?



In [ ]:

# plot decision boundaries
# TODO

# plot margins left and right, you can use linestyle='dashed'
# TODO


# plot datasets
# TODO


- 1.4 Lets compute the test error of each hyperplane. Define first a function `clf_predict` that predicts the decisions for a given classifier: $\hat{y}({x})=\operatorname{sign}(\langle{w}, {x}\rangle+b)$. The function takes as input weight $w$ and bias $b$ of the hyperplane and features `X`. Use the `test_error` function you defined above to compute the test error for Classifiers A, B and C. Are the results in line with your observation?

In [ ]:
def clf_predict(w,b,X):
    # TODO

In [ ]:
# Print test error for A, B, C
# TOODO

## Hard-margin Support Vector Machine
We formalize a hard-margin SVM as
\begin{array}{l}
\min _{w \in \mathbb{R}^{d}, b \in \mathbb{R}} \frac{1}{2}\|w\|^{2} \\
\text { subject to: } y_{i}\left(\left\langle w, x_{i}\right\rangle+b\right) \geq 1, \quad \forall i=1, \ldots, n
\end{array}

The support vectors are the points on the margin, that is $\langle w, x\rangle+b=\pm 1$.The area between the two supporting hyperplanes $\{x \mid\langle w, x\rangle+b=1\}$ and $\{x \mid\langle w, x\rangle+b=-1\}$ is called the margin.  

- 1.5 Analyze the performance of a linear SVM. For simplicity you can use [this](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model. Use a default value of `C=1000`. Compute the test error and plot the classification boundary. How many support vectors do you find? Can you print them out?

In [ ]:
from sklearn.svm import SVC

# fit model
# TODO

# compute test error
# TODO

# plot dataset
# TODO

# plot decision boundary
# TODO


1.6 In the lecture, we learned that modifications of the training points matter only if they fall into the margin / or on the wrong side of the decision boundary. Lets try that. Let us use dataset A. Find a datapoint for each of the following cases and augment the the dataset A with the datapoint. Analyze the performance of a linear SVM with `C=1000` on both the original and the augmented dataset. Compute the test error and plot the classification boundary with the support vectors.
- POINT A: correctly classified, outside margin
- POINT B: exactly on margin
- POINT C: correctly classified, inside margin
- POINT D: misclassified, inside margin
- POINT E: misclassified, outside margin

In [ ]:
# Original dataset

# Fit model
# TODO

# Compute test error
# TODO

In [ ]:
# For Points A, B, C, D, E:

## Define point and concatenate to X_blobs1 and y_blobs1
## TODO

## fit model on augmented dataset
## TODO

## compute test error
## TODO

In [ ]:
# plot original dataset and the 5 different augmented datasets and decision boundaries
# TODO

# optional: colour the added point in green for visibility
# TODO


# 2. Not linearly seperable data
Previously we have defined a hard-margin SVM. We now move to dataset B and explore, why is this problematic and what a soft-margin SVM does.

## Soft-margin Support Vector Machine
We formalize a soft-margin SVM as
$$
\begin{aligned}
\min _{\mathbf{w} \in \mathbb{R}^{d}, b \in \mathbb{R}, \boldsymbol{\xi} \in \mathbb{R}^{n}} & \frac{1}{2}\|\mathbf{w}\|^{2}+\frac{C}{n} \sum_{i=1}^{n} \xi_{i} \\
\text { subject to: } & Y_{i}\left(\left\langle\mathbf{w}, \mathrm{x}_{i}\right\rangle+b\right) \geq 1-\xi_{i}, \quad \forall i=1, \ldots, n, \\
& \xi_{i} \geq 0, \quad \forall i=1, \ldots, n
\end{aligned}
$$

- 1.7 What is the functionality of parameter `C`? Why did we choose in the previous example `C=1000`? Explore different values of the `C` parameter `[0.001, 0.01, 1, 10, 100, 1000]` for a linear SVM. Compute the test error and plot the classification boundary for each parameter value. Why did we not include `C=0`?

In [ ]:
# plot original dataset B
# TODO

In [ ]:
# for the C values given above
## fit model
## TODO

## compute test error
## TODO

## plot dataset
## TODO

## plot decision boundary
## TODO

# 3. Non-linear problems
Until now, we have seen linear separable and non-separable problems. We solved with a linear SVM with a soft margin. A linear SVM uses a kernel: $\left\langle x, x^{\prime}\right\rangle$. We now move to dataset C and non-linear problems. 

- 3.1 Analyze the performance of a linear SVM with value `C = 20` on dataset C. Compute the test error and plot the classification boundary and support vectors. What do you observe?

In [ ]:
# fit the model 
# TODO

# compute the test error
# TODO

# plot the dataset
# TODO

# plot the decision boundary
# TODO


## Radial Basis Function (RBF) Kernel
We can use an SVM for non-linear problems by choosing a kernel that fits to our data. The radial basis function (RBF) kernel is defined as $$\exp \left(-\gamma\left\|x-x^{\prime}\right\|^{2}\right),$$ where $\gamma$ is specified by parameter gamma and must be greater than 0.

- 3.2  Before we move to solving this non-linear problem with an SVM, lets compute a radial basis function centered on the middle clump: `r = np.exp(-(X ** 2).sum(1))`. Visualize this extra data dimension using a three-dimensional plot. HINT: Running the notebook live, you will be able to rotate the plot.

In [ ]:
# code the radial basis function given above
# TODO

# concatenate it as third dimension to the dataset
# TODO

# plot the data with a 3D plot, colour one class in red, the other in blue
# TODO

- 3.3 Train a no-linear SVM with a RBF kernel. Set `C=1` and explore different values of the `gamma` parameter `[0.01, 0.1, 1, 10, 100]`. Compute the test error and plot the classification boundary. Set `gamma=1` and explore different values of the `C` parameter `[0.01, 0.1, 1, 10, 100]`. Compute the test error and plot the classification boundary. - What does a parameter `C<0`, `C=0` and `C>1` mean?

In [ ]:
# for each value in gamma

## fit the model 
## TODO

## compute the test error
## TODO

## plot the dataset
## TODO

## plot the decision boundary
## TODO

In [ ]:
# for each value in C

## fit the model 
## TODO

## compute the test error
## TODO

## plot the dataset
## TODO

## plot the decision boundary
## TODO

# 4. Choosing Parameters
One of the limitations of SVMs is that there is the complexity parameter `C` - and for non-linear kernels a parameter previously called `gamma` - that must be found using a hold-out method such as cross-validation (CV). Lets take a look at this together.
- Use a 5-fold CV (via `cross_val_score`, see [here](https://scikit-learn.org/stable/modules/cross_validation.html)) to adjust both `C` and `gamma` parameters of a RBF SVM for not linearly seperable data. Explore values of `C` in `[0.01, 0.01, 0.1, 1, 10, 100, 1000]`, and define the range of `gamma` as: `[0.125, 0.25, 0.5, 1, 2, 4, 8])/D`, being `D` the data dimension. Note that this definition of the `gamma` values is used alleviate the influence of the data dimension in the definition of the RBF kernel. Print mean and standard deviation of the scores. Which model provides you the best results? 

In [ ]:
# for each combination of C and gamma

## compute cross validation score 
## TODO

## print mean and standard deviation
## TODO



HINT: For further information on the parameters and their trade-off, take a look [here](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html).

# 5. Different Kernels, Multiple Classes 
In addition to the linear and RBF kernel, scikit-learn also implements e.g. the following kernels
- polynomial: $\left(\gamma\left\langle x, x^{\prime}\right\rangle+r\right)^{d}$, where $d$ is specified by parameter `degree`, $r$ by `coef0`.
-  sigmoid: $\tanh \left(\gamma\left\langle x, x^{\prime}\right\rangle+r\right)$,  where  $r$  is specified by `coef0`.

We will take a brief look at what how their decision boundaries look like.

- Lets move to a multi-class dataset D and take a look at the decision boundaries of the different kernels. Analyze the performance of an SVM with `linear`, `rbf`(gamma=0.7),`poly`(with `degree=3`, `gamma='auto'`) and `sigmoid`(`gamma='auto'`) kernels. For the sake of simplicity, you can use a default value of `C = 1`. Compute the test error and plot the classification boundary. What does the gamma mean for each of the kernels? 

In [ ]:
# from here: https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html

def make_meshgrid(x, y, h=.02):
    # Create a mesh of points to plot in
    # from 
    
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(ax, clf, xx, yy, **params):
    #Plot the decision boundaries for a classifier.
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out


In [ ]:
# define the four SVC models
## TODO

# for each model
## fit
## TODO

## compute test error
## TODO


# from here: https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html
fig, sub = plt.subplots(2, 2,figsize=(15,15))
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X_blobs3[:, 0], X_blobs3[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, errors, ax in zip(models, titles, errors, sub.flatten()):
    plot_contours(ax, clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y_blobs3, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('x_1')
    ax.set_ylabel('x_2')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title + ", Error " + str(er))

plt.show()

HINT: Not enough kernels to choose from? Interested in a DIY-kernel? Check [this](https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html#sphx-glr-auto-examples-svm-plot-custom-kernel-py) out.

# 6. Further Considerations (self-study)
The following considerations are not study focus of this notebook. 
- unbalanced classes: [here](https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html#sphx-glr-auto-examples-svm-plot-separating-hyperplane-unbalanced-py)
- weighted samples: [here](https://scikit-learn.org/stable/auto_examples/svm/plot_weighted_samples.html#sphx-glr-auto-examples-svm-plot-weighted-samples-py)
- one-versus-rest multi-class approach: [here](https://scikit-learn.org/stable/auto_examples/svm/plot_iris_svc.html#sphx-glr-auto-examples-svm-plot-iris-svc-py)
- SVMs on hand written digits: [here](https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html)
